In [13]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 

# from numba import jit
import matplotlib.pyplot as plt
import mplfinance as mpf
import talib as ta
import tushare as ts
# ts.set_token('11137efdbeac800606d677871a55b3fd5aef79890c59867a8f34d03e')
# pro = ts.pro_api()

df=pd.DataFrame()  # 初始化空值
vTest = 1000

from importlib import reload 
from qflib import basic,caldata
reload(basic)
reload(caldata)

engine = basic.engine()
# data_dir = '~/qfdata/'
data_dir = '../'

# sl_list = pd.read_csv( '../data/sl_hs_const.csv')
sl_list = pd.read_csv( '../data/sl_shengutong.csv')
# sl_list = pd.read_csv( '../data/sl_company.csv')
# sl_list.to_csv( '../data/stocklist.csv', index=False)
# sl_list = pd.read_csv( data_dir+'data/stocklist.csv')
sl_list.shape[0]

# from numba import jit

''' 清空 stock_cal table '''
# sql = "delete from stock_cal where true"
sql = "truncate table stock_cal "
basic.exec_sql(engine, sql)


In [14]:
%%time

print('计算各类指标 : ')
for i in range( len(sl_list)  ):
    
    print(sl_list.index[i],'-', sl_list['ts_code'][i], end=" ")
    
    # 1、国内股市：可显示复权后和复权前信息, 现在用前复权价格
    # df = ts.pro_bar(ts_code=sl_list['ts_code'][i], asset='E', adj='qfq', start_date=start_date, end_date='')
    # 2、本地MySQL读取
    sql = "select * from tran_daily where ts_code='"+sl_list['ts_code'][i]+"'"
    df = basic.read_data(engine, sql)
    df.set_index('trade_date', drop=True, append=False, inplace=True)
    # df = pd.read_csv( ts_file, index_col='trade_date', parse_dates=True).sort_index(ascending=True)

    if len(df) == 0:
        continue

    # 字段改名
    # df = df.sort_index(ascending=True)
    df.rename( columns={'vol':'volume','pct_chg':'pct_change', 'amount':'value'}, inplace=True  )
    # 实体部分细化
    df['pct_upper'] = df.apply(caldata.cal_upper, axis=1)   # 上引线百分比
    df['pct_down']  = df.apply(caldata.cal_down,  axis=1)   # 下引线百分比
    df['pct_range'] = df.apply(caldata.cal_range, axis=1)   # 振幅百分比
    df['st_high']   = df.apply(caldata.cal_st_high,  axis=1)   # 实体高点价格
    df['st_low']    = df.apply(caldata.cal_st_low,   axis=1)   # 实体低点价格
    # df['pct_st']  = df.apply(caldata.cal_st, axis=1)   # 实体高度
    df['pct_st_range']  = df.apply(caldata.cal_st_range, axis=1)   # 实体部分百分比
    # 星线定义

    # df.to_csv( data_dir+'/tushare/' + sl_list['ts_code'][i] +'.ts.csv', index=False )
    print('.d', end="")
    
    # for i in range( sl_list.shape[0]  ):
    # ？？， 不计算8字头，北京股票
    # print(sl_list.index[i], sl_list['ts_code'][i]," ",end="")
    if sl_list['ts_code'][i].startswith('8'):
        # break
        continue
    
    # if len(df) > 0:
    # 计算MACD
    ''' way1 : charlie '''
    # df = cal_macd(df, 2, 12, 26, 9)
    ''' way2 : ta-lib '''
    close = df.close.values  # 为 MACD 数据初始化close
    df['diff'], df['dea'], df['bar'] = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    
    # 计算MA
    ''' way1: charlie  '''
    # df = cal_ma(df)
    
    ''' way 2: ta-lib '''
    # df['MA15_pct_change'] =df['pct_change'].rolling(15).mean() 
    df.insert( df.shape[1], 'MA5', ta.MA(df['close'], 5) )
    df.insert( df.shape[1], 'MA10', ta.MA(df['close'], 10) )
    df.insert( df.shape[1], 'MA20', ta.MA(df['close'], 20) )
    df.insert( df.shape[1], 'MA30', ta.MA(df['close'], 30) )
    df.insert( df.shape[1], 'MA60', ta.MA(df['close'], 60) )
    df.insert( df.shape[1], 'MA125', ta.MA(df['close'], 125) )
    df.insert( df.shape[1], 'MA250', ta.MA(df['close'], 250) )
    # for BT
    # 股价变化均值
    df.insert( df.shape[1], 'MA3_pct_change', ta.MA(df['pct_change'], 3) )
    df.insert( df.shape[1], 'MA5_pct_change', ta.MA(df['pct_change'], 5) )
    df.insert( df.shape[1], 'MA10_pct_change', ta.MA(df['pct_change'], 10) )
    df.insert( df.shape[1], 'MA15_pct_change', ta.MA(df['pct_change'], 15) )
    df.insert( df.shape[1], 'MA30_pct_change', ta.MA(df['pct_change'], 30) )
    # N天之内最大股价
    df['MAX3_pct_change']=df['pct_change'].rolling(3).max()
    df['MAX5_pct_change']=df['pct_change'].rolling(5).max()
    df['MAX10_pct_change']=df['pct_change'].rolling(10).max()
    df['MAX15_pct_change']=df['pct_change'].rolling(15).max()
    df['MAX30_pct_change']=df['pct_change'].rolling(30).max()
    # n天之内最高收盘价
    df['MAX90_high']=df['high'].rolling(90).max()
    df['MAX30_high']=df['high'].rolling(30).max()
    df['MAX3_high']=df['high'].rolling(3).max()
    
    ''' 计算BOLL '''
    df['bb-u'], df['bb-m'], df['bb-l'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )
    df['upper'], df['middle'], df['lower'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )
    
    ''' 计算是否星线 '''
    df['xing'] = df['pct_st_range']<=1

    # 空值填充为0
    df = df.fillna(value=0)

    """ 输出 to csv """
    df.to_csv( data_dir+'cal/' + sl_list['ts_code'][i] +'.cal.csv', index=True )
    
    ''' 输出到 mysql '''
    sql = "delete from stock_cal where ts_code='"+sl_list['ts_code'][i] +"'"
    basic.exec_sql(engine, sql)
    df.to_sql('stock_cal', engine, index=True, if_exists='append', chunksize=5000)
 
    # 首次运行，建立stock_cal表, 可以从df中建立表，表之前不存在，然后再改字段和index
    # basic.write_data(engine,df,'stock_cal2', True)

    # print( '.s; ', end='')
    print( '.s; ')

print('\n\n指标运算结束')



计算各类指标 : 
0 - 002910.SZ .d.s; 
1 - 000016.SZ .d.s; 
2 - 001872.SZ .d.s; 
3 - 000040.SZ .d.s; 
4 - 000401.SZ .d.s; 
5 - 000429.SZ .d.s; 
6 - 000553.SZ .d.s; 
7 - 000560.SZ .d.s; 
8 - 000710.SZ .d.s; 
9 - 000723.SZ .d.s; 
10 - 000951.SZ .d.s; 
11 - 000968.SZ .d.s; 
12 - 002061.SZ .d.s; 
13 - 002507.SZ .d.s; 
14 - 002530.SZ .d.s; 
15 - 002537.SZ .d.s; 
16 - 002545.SZ .d.s; 
17 - 002611.SZ .d.s; 
18 - 002648.SZ .d.s; 
19 - 002747.SZ .d.s; 
20 - 002812.SZ .d.s; 
21 - 002839.SZ .d.s; 
22 - 002841.SZ .d.s; 
23 - 002859.SZ .d.s; 
24 - 002867.SZ .d.s; 
25 - 300308.SZ .d.s; 
26 - 300457.SZ .d.s; 
27 - 300616.SZ .d.s; 
28 - 300618.SZ .d.s; 
29 - 300628.SZ .d.s; 
30 - 300666.SZ .d.s; 
31 - 002936.SZ .d.s; 
32 - 000301.SZ .d.s; 
33 - 000537.SZ .d.s; 
34 - 000546.SZ .d.s; 
35 - 000789.SZ .d.s; 
36 - 000831.SZ .d.s; 
37 - 000932.SZ .d.s; 
38 - 000935.SZ .d.s; 
39 - 001965.SZ .d.s; 
40 - 002258.SZ .d.s; 
41 - 002382.SZ .d.s; 
42 - 002409.SZ .d.s; 
43 - 002430.SZ .d.s; 
44 - 002511.SZ .d.s; 
45 - 00267

In [15]:
# basic.write_data(engine, df, 'stock_basic', False, 'append')
# basic.write_data(engine, df, 'stock_cal', True, 'replace')

In [16]:

# 插入和删除行，很有用
# df.insert( df.shape[1], 'MA250', ta.MA(df['close'], 250) )
# df.drop( labels=['MA125'], axis=1, inplace=True)